In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from scipy.stats import zscore

# Ensure the data folder exists and the file is present
data_path = '../data/benin-malanville.csv'
if not os.path.exists(data_path):
    raise FileNotFoundError(f"File not found at {data_path}. Ensure 'benin-malanville.csv' is in the 'data/' folder.")

# Load the dataset
df_benin = pd.read_csv(data_path)

# Task 2: Data Profiling, Cleaning & EDA

# 1. Summary Statistics & Missing-Value Report
print("Summary Statistics:")
summary_stats = df_benin.describe()
print(summary_stats)

print("\nMissing Values:")
missing_values = df_benin.isna().sum()
print(missing_values)

# Calculate percentage of missing values
missing_percentage = (df_benin.isna().sum() / len(df_benin)) * 100
print("\nColumns with >5% Missing Values:")
print(missing_percentage[missing_percentage > 5])

# 2. Outlier Detection & Basic Cleaning
# Compute Z-scores for key columns
key_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
df_benin_zscores = df_benin[key_columns].apply(zscore, nan_policy='omit')
outliers = (df_benin_zscores.abs() > 3).any(axis=1)
print(f"\nNumber of rows with outliers (|Z| > 3): {outliers.sum()}")

# Impute missing values in key columns with median
for col in key_columns:
    df_benin[col].fillna(df_benin[col].median(), inplace=True)

# Drop rows with missing values in other critical columns (if any)
df_benin.dropna(subset=['Tamb', 'RH', 'BP'], inplace=True)

# Export cleaned DataFrame
cleaned_path = '../data/benin_clean.csv'
df_benin.to_csv(cleaned_path, index=False)
print(f"\nCleaned DataFrame exported to {cleaned_path}")

# 3. Time Series Analysis
# Convert Timestamp to datetime
df_benin['Timestamp'] = pd.to_datetime(df_benin['Timestamp'])

# Plot GHI, DNI, DHI, Tamb vs. Timestamp
plt.figure(figsize=(12, 8))
for col in ['GHI', 'DNI', 'DHI', 'Tamb']:
    plt.plot(df_benin['Timestamp'], df_benin[col], label=col)
plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.title('Time Series of GHI, DNI, DHI, and Tamb')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Monthly trends
df_benin['Month'] = df_benin['Timestamp'].dt.month
monthly_avg = df_benin.groupby('Month')[['GHI', 'DNI', 'DHI']].mean()
monthly_avg.plot(kind='bar', figsize=(10, 6))
plt.title('Monthly Average GHI, DNI, DHI')
plt.xlabel('Month')
plt.ylabel('Average Value (W/m²)')
plt.tight_layout()
plt.show()

# 4. Cleaning Impact
cleaning_impact = df_benin.groupby('Cleaning')[['ModA', 'ModB']].mean()
cleaning_impact.plot(kind='bar', figsize=(8, 6))
plt.title('Average ModA and ModB Pre/Post Cleaning')
plt.xlabel('Cleaning (0 = No, 1 = Yes)')
plt.ylabel('Average Value (W/m²)')
plt.tight_layout()
plt.show()

# 5. Correlation & Relationship Analysis
# Correlation heatmap
corr_matrix = df_benin[['GHI', 'DNI', 'DHI', 'TModA', 'TModB']].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.show()

# Scatter plots
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.scatter(df_benin['WS'], df_benin['GHI'], alpha=0.5)
plt.xlabel('Wind Speed (m/s)')
plt.ylabel('GHI (W/m²)')
plt.title('WS vs. GHI')

plt.subplot(1, 3, 2)
plt.scatter(df_benin['WSgust'], df_benin['GHI'], alpha=0.5)
plt.xlabel('Wind Gust (m/s)')
plt.title('WSgust vs. GHI')

plt.subplot(1, 3, 3)
plt.scatter(df_benin['RH'], df_benin['Tamb'], alpha=0.5)
plt.xlabel('Relative Humidity (%)')
plt.ylabel('Tamb (°C)')
plt.title('RH vs. Tamb')
plt.tight_layout()
plt.show()

# 6. Wind & Distribution Analysis
# Histogram for GHI
plt.figure(figsize=(8, 6))
plt.hist(df_benin['GHI'], bins=30, edgecolor='black')
plt.title('Histogram of GHI')
plt.xlabel('GHI (W/m²)')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

# Histogram for WS
plt.figure(figsize=(8, 6))
plt.hist(df_benin['WS'], bins=30, edgecolor='black')
plt.title('Histogram of Wind Speed')
plt.xlabel('WS (m/s)')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

# 7. Temperature Analysis
plt.figure(figsize=(8, 6))
plt.scatter(df_benin['RH'], df_benin['GHI'], alpha=0.5)
plt.xlabel('Relative Humidity (%)')
plt.ylabel('GHI (W/m²)')
plt.title('RH vs. GHI')
plt.tight_layout()
plt.show()

# 8. Bubble Chart
plt.figure(figsize=(10, 6))
plt.scatter(df_benin['Tamb'], df_benin['GHI'], s=df_benin['RH']*10, alpha=0.5)
plt.xlabel('Ambient Temperature (°C)')
plt.ylabel('GHI (W/m²)')
plt.title('GHI vs. Tamb (Bubble Size = RH)')
plt.tight_layout()
plt.show()

# Note: Wind rose plot requires additional libraries (e.g., windrose). Install if needed:
# pip install windrose
# Example wind rose (uncomment if installed):
"""
from windrose import WindroseAxes
ax = WindroseAxes.from_ax()
ax.bar(df_benin['WD'], df_benin['WS'], normed=True, opening=0.8, edgecolor='white')
ax.set_legend()
plt.title('Wind Rose Plot')
plt.show()
"""

Summary Statistics:
                 GHI            DNI            DHI           ModA  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      240.559452     167.187516     115.358961     236.589496   
std       331.131327     261.710501     158.691074     326.894859   
min       -12.900000      -7.800000     -12.600000       0.000000   
25%        -2.000000      -0.500000      -2.100000       0.000000   
50%         1.800000      -0.100000       1.600000       4.500000   
75%       483.400000     314.200000     216.300000     463.700000   
max      1413.000000     952.300000     759.200000    1342.300000   

                ModB           Tamb             RH             WS  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      228.883576      28.179683      54.487969       2.121113   
std       316.536515       5.924297      28.073069       1.603466   
min         0.000000      11.000000       2.100000       0.000000   
25%         0

C:\Users\User\AppData\Local\Temp\ipykernel_4156\2847870618.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_benin[col].fillna(df_benin[col].median(), inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_4156\2847870618.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


OSError: Cannot save file into a non-existent directory: 'data'